In [1]:
import sys
sys.path.insert(0, '/home/christo/notebooks/christo/prod/')
sys.path.insert(0, '/home/christo/notebooks/christo/company_maintenance_scripts/')
import re
import pandas as pd
from NedbankSpendPy import  client
import time
from datetime import datetime 

In [2]:
from nedbank_companies_addition_lib import maxindexfn, mongo_regex_searchstring
from nedbank_companies_addition_lib import nedbankcompany_class_check, classlookupfn, input_dict_fn, mongo_regex_searchstring_row
from nedbank_companies_addition_lib import  nedbank_companies_db, regex_searchstring

In [3]:
def franchise_agg_fn(collection):
    return pd.DataFrame(
    list(collection.aggregate([{'$match':
                                                        {
                                                            'companyname':
                                                            {
                                                                '$exists':True
                                                            }
                                                        }
                                                       },
                                                       {'$project':{
                                                           'companyname':1,
                                                           'franchisename':1,
                                                           'Dedupegroup':1,
#                                                            'companyindex':1,
                                                           'TransactionAmount':1}},
                                                       {
                                                           '$group':
                                                           {
                                                               '_id':
                                                               {
                                                                   'companyname':'$companyname',
                                                                   'franchisename':'$franchisename'
                                                               },
                                                               "uniqueclientlist":{"$addToSet":'$Dedupegroup'},
                                                               "transactioncount":
                                                               {
                                                                   '$sum': 1
                                                               },
                                                               "transactionamount":
                                                               {
                                                                   '$sum':'$TransactionAmount'
                                                               }
                                                           }
                                                       },
                                                       {'$project':{'_id':0,'companyname':'$_id.companyname',
                                                                    'franchisename':'$_id.franchisename',
#                                                                     'companyindex':'$_id.companyindex',
                                                                    'n_uniqueclients':{'$size': "$uniqueclientlist"},
                                                                   'transactioncount':1,
                                                                   'transactionamount':1}}
                                                      ], allowDiskUse=True)))



In [4]:
collectionlist=['clientswipes_201701','clientswipes_201702','clientswipes_201703',
                'clientswipes_201704','clientswipes_201705','clientswipes_201706',
                'clientswipes_201707','clientswipes_201708','clientswipes_201709',
                'clientswipes_201710','clientswipes_201711','clientswipes_201712',
                'clientswipes_201801','clientswipes_201802','clientswipes_201803',
                'clientswipes_201804','clientswipes_201805','clientswipes_201806',
                'clientswipes_201807','clientswipes_201808','clientswipes_201809',
                'clientswipes_201810','clientswipes_201811','clientswipes_201812',                
                'clientswipes_201901','clientswipes_201902','clientswipes_201903',
                'clientswipes_201904','clientswipes_201905','clientswipes_201906',
                'clientswipes_201907','clientswipes_201908','clientswipes_201909',
                'clientswipes_201910','clientswipes_201911','clientswipes_201912'
               'clientswipes_202001']

### Create a NEW pivot in aggregated_franchise_spend for ALL the periods listed above:

In [ ]:
write_collection = client.lookups['aggregated_franchise_spend']
dn=datetime.now()
m=dn.month
d=dn.day
y=dn.year
savestr= f'{y:0}'+f'{m:02}'+f'{d:02}'
# df=pd.DataFrame()
for collectionname in collectionlist:
#     print('-----------------------------------------------------------------------------')
    print('===== now doing collection: {collectionname} ====================================='.format(collectionname=collectionname))
    period=collectionname[-6:]
    print('Doing period: {period}'.format(period=period))
    read_collection=client.behaviourdb[collectionname]
    print('Collect data:')    
    agg_df=franchise_agg_fn(read_collection)
    print('Set index:')    
#     agg_df.set_index(['companyindex','companyname'],inplace=True)
    print('Rename columns:')        
    agg_df.rename(columns={'transactionamount':'amount_{period}'.format(period=period),
                      'transactioncount':'count_{period}'.format(period=period),
                          'n_uniqueclients':'n_uniqueclients_{period}'.format(period=period)}, inplace=True)
#     if df.empty:
#         print('The history is still empty!')
#         df=agg_df.copy()
#     else:
#         print('Merge new data with (non-empty) history:')        
#         df=df.merge(agg_df, right_index=True,left_index=True, how='outer')
    print('Save (new) history:')
#     agg_df.reset_index(inplace=True)
    agg_df['extracted_date']=savestr
    write_collection.insert_many(agg_df.to_dict('records'))
#     df.to_csv(df.to_csv('/home/christo/data/companydb_franchiselevel_aggregated_pivot_{savestr}.csv'.format(savestr=savestr)))
    

### Investigate companies from the franchise pivot collection:

In [5]:
read_collection = client.lookups['aggregated_franchise_spend']

In [10]:
df=pd.DataFrame(list(read_collection.find({'companyname':'BP', 'amount_201912':{'$exists':True}},{'_id':0})))

In [11]:
df.sort_values(["n_uniqueclients_201912"], ascending = False)

,amount_201912,companyname,count_201912,extracted_date,franchisename,n_uniqueclients_201912
1447,-1022944.54,BP,3068,20200305,BP RIDGE OAS,2613
159,-1129521.13,BP,3216,20200305,BP AIRPORT,2479
228,-1046274.23,BP,2963,20200305,BP BEYERS NA,2395
1178,-1161959.02,BP,2121,20200305,BP MONTE,1756
336,-1026261.93,BP,2166,20200305,BP CAPE TOWN,1505
655,-1162918.13,BP,2586,20200305,BP FAIRWAYS,1442
576,-847796.88,BP,1961,20200305,BP EDENVALE,1416
1746,-962080.57,BP,2146,20200305,BP WATERFALL,1395
1357,-992890.91,BP,1942,20200305,BP PINELANDS,1331
1311,-1077534.21,BP,2076,20200305,BP OLIVEDALE,1296


In [ ]:
read_collection = client.lookups['aggregated_franchise_spend']
df=pd.DataFrame(list(read_collection.distinct('extracted_date')))

In [ ]:
df[0][0]

In [ ]:
collection=client.behaviourdb['clientswipes_201701']
df=pd.DataFrame(list(collection.distinct('franchisename',{'companyname':'BOLT'})))

In [ ]:
list(df)

In [ ]:
flist=list(df[0])

In [ ]:
set(flist)

In [ ]:
collection=client.lookups['nedbank_companies']
df=pd.DataFrame(list(collection.find({'companyname':'BOLT'})))

In [ ]:
avoidlists=df.avoidlists[0]
aliases=df.aliases[0]

In [ ]:
{1,2, 7,8,9,10}-{1,2,3,4,5,6}

In [ ]:
aliases_set=set(aliases)
aliases_set.intersection(set(avoidlists))

In [ ]:
list(set(flist))

In [ ]:
set(flist)-set(avoidlists)

In [ ]:
avoidlists.sort()

In [ ]:
avoidlists

In [ ]:
flist.sort()
flist